<a href="https://colab.research.google.com/github/mkierczak/autoencoders_workshop/blob/main/PCA2VAE_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we will download and upack data accompanying article by Lazaridis et. al. [*Genomic insights into the origin of farming in the ancient Near East*](https://www.nature.com/articles/nature19310) 2016. Nature **536**:419-424.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
from keras import layers
from keras import backend as K
from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model
from keras import backend
from sklearn.decomposition import PCA
from itertools import product

In [1]:
!wget -O data_hd5 https://www.dropbox.com/scl/fi/kck4puyi1qmuzr65bgdbn/HumanOriginsPublic2068_geno_chr1.hd5?rlkey=xp4nfljz0c2za9ihriletxx3x&dl=0

--2023-09-01 11:02:54--  https://www.dropbox.com/scl/fi/kck4puyi1qmuzr65bgdbn/HumanOriginsPublic2068_geno_chr1.hd5?rlkey=xp4nfljz0c2za9ihriletxx3x
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/e/scl/fi/kck4puyi1qmuzr65bgdbn/HumanOriginsPublic2068_geno_chr1.hd5?rlkey=xp4nfljz0c2za9ihriletxx3x [following]
--2023-09-01 11:02:54--  https://www.dropbox.com/e/scl/fi/kck4puyi1qmuzr65bgdbn/HumanOriginsPublic2068_geno_chr1.hd5?rlkey=xp4nfljz0c2za9ihriletxx3x
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccf366adfaa53e80cc43ba93e50.dl.dropboxusercontent.com/cd/0/inline/CC6vmH2EYD9q210-t5ErlsblUkyNwq7yJuafnfL1wjFbtixGFR8qu1oE_XxbX77HXcR--wQePgxXbt7tNeRZbvNKRSMR0sLz1AFOmSoR9pNm9aFTMYR-nYN_e9ECp52YJxA/file# [following]
--2

In [7]:
orig_gt = pd.read_hdf('./data_hd5', key = 'geno')
orig_pheno = pd.read_hdf('./data_hd5', key = 'pheno')
orig_snp = pd.read_hdf('./data_hd5', key = 'snps')


In [26]:
print(orig_gt.info())
print(orig_gt.iloc[0:4, 0:4])
print(orig_pheno.iloc[0:4, ])
print(orig_snp.iloc[0:4, ])

<class 'pandas.core.frame.DataFrame'>
Index: 2068 entries, SA1004 to G434
Columns: 48433 entries, rs7419119 to rs10788875
dtypes: float64(48433)
memory usage: 764.2+ MB
None
        rs7419119  rs13302957  rs6696609  rs8997
SA1004        2.0         0.0        1.0     2.0
SA063         0.0         2.0        1.0     1.0
SA010         2.0         1.0        0.0     1.0
SA064         2.0         1.0        1.0     1.0
       id gender      pop
0  SA1004      F  Khomani
1   SA063      F  Khomani
2   SA010      F  Khomani
3   SA064      M  Khomani
          SNP CHR        cm       POS A1 A2
0   rs7419119   1  0.022518  842013.0  T  G
1  rs13302957   1  0.024116  891021.0  G  A
2   rs6696609   1  0.024457  903426.0  C  T
3      rs8997   1  0.025727  949654.0  A  G


In [30]:
print("Missing genotypes per marker: \n", orig_gt.isna().sum())

Missing genotypes per marker: 
 rs7419119      4
rs13302957     3
rs6696609     14
rs8997        10
rs9442372      5
              ..
rs6683472      6
rs4926509     10
rs11205415     7
rs4072273      4
rs10788875     5
Length: 48433, dtype: int64


In [31]:
geno = orig_gt.replace([0, 1.0, 2.0], [0.5, 0.75, 1.0])
geno.fillna(0, inplace = True)
print(geno.isna().sum())
print(geno.iloc[0:5, 0:5])


rs7419119     0
rs13302957    0
rs6696609     0
rs8997        0
rs9442372     0
             ..
rs6683472     0
rs4926509     0
rs11205415    0
rs4072273     0
rs10788875    0
Length: 48433, dtype: int64
        rs7419119  rs13302957  rs6696609  rs8997  rs9442372
SA1004       1.00        0.50       0.75    1.00       0.50
SA063        0.50        1.00       0.75    0.75       0.75
SA010        1.00        0.75       0.50    0.75       0.50
SA064        1.00        0.75       0.75    0.75       0.75
SA073        0.75        0.50       0.75    0.75       0.50


In [41]:
train = geno.sample(frac = 0.8, random_state = 42)
test = geno.drop(train.index)
pheno = orig_pheno.set_index(keys = 'id')
train_pheno = pheno[pheno.index.isin(train.index)]
test_pheno = pheno.drop(train.index)
train.reset_index()
test.reset_index()
train_pheno.reset_index()
test_pheno.reset_index()

# Print some info about the resulting split
print("Original data:", orig_gt.shape)
print("\t - training set:", train.shape)
print("\t - test set:", test.shape)

Original data: (2068, 48433)
	 - training set: (1654, 48433)
	 - test set: (414, 48433)
